<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2023/blob/main/ThreeTanks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import numpy as np
import jax.numpy as jnp
import jax
jax.config.update('jax_enable_x64',True)
from scipy.integrate import solve_ivp
from plotly.subplots import make_subplots

In [78]:
Ac=1.
Ah = 0.3
g=9.81
qin0 = 0.1

tend=10

h10 = ((qin0/Ah)**2)/2/g
h20 = h10
h30 = h10

hsp = 1.2 * h30
Kc = 29.7  #(0.45 * Ku)
errint0 = 0.
taui = 0.366
taud= 0.5
# taui=1e15
#Ku = 66

In [80]:
def rhs3tank(t, vec):
    h1, h2, h3, errint = vec
    err = hsp - h3
    q1, q2, q3 = Ah*jnp.sqrt(2*g)*jnp.sqrt( jnp.where(vec[:3]>0, vec[:3], 0.))
    qin = qin0 + Kc*(err + errint/taui - taud*(q2-q3)/Ac)

    dh1 = (qin - q1)/Ac
    dh2 = (q1 - q2)/Ac
    dh3 = (q2 - q3)/Ac
    return jnp.array([dh1, dh2, dh3, err])

def rhs1tank(t, vec):
    h1, errint = vec
    err = hsp - h1
    qin = qin0 + Kc*(err + errint/taui)
    q1 = Ah*np.sqrt(2*g)*np.sqrt(vec[0])
    dh1 = (qin - q1)/Ac
    return np.array([dh1, err])

In [81]:
rhs3tank_jit =jax.jit(rhs3tank)
rhs3tank_jac=jax.jit(jax.jacobian(rhs3tank, 1))

In [82]:
res=solve_ivp(rhs3tank_jit, (0,tend), np.array([h10, h20, h30, errint0]), method='Radau', dense_output=True,
              rtol=1e-9, atol=1e-12, jac=rhs3tank_jac)
# res=solve_ivp(rhs1tank, (0,tend), np.array([h10, errint0]), method='Radau', dense_output=True)


In [56]:
# tplot = np.linspace(0,tend,1000)
# h1plot, errintplot =res.sol(tplot)
# fig=make_subplots()
# fig.add_scatter(x=tplot,y=h1plot,mode='lines',name='h1')

In [83]:
tplot = np.linspace(0,tend,1000)
h1plot,h2plot,h3plot, errintplot =res.sol(tplot)
fig=make_subplots()
fig.add_scatter(x=tplot,y=h1plot,mode='lines',name='h1')
fig.add_scatter(x=tplot,y=h2plot,mode='lines',name='h2')
fig.add_scatter(x=tplot,y=h3plot,mode='lines',name='h3')
fig.update_layout(template='plotly_dark')